In [1]:
%load_ext beam_setup

Setting up the Beam environment for interactive use
Standard modules will be automatically imported so you can use them without explicit import
Done importing packages. It took:  4.2 seconds
Beam library is loaded from path: /home/elad/docker/beamds/src/beam
The Beam version is: 2.5.11


In [7]:
from beam.auto import AutoBeam
from beam.serve import beam_server
from beam.serve import BeamServeConfig

## Build Your Algorithm

For the purpose of this demo we will use a fake algorithm that generates some random data

In [3]:
from beam.misc import BeamFakeAlg
fake_alg = BeamFakeAlg(sleep_time=1., variance=0.5, error_rate=0.)

An example interaction with the algorithm

In [4]:
fake_alg.run('hi')

{'name': 'Linda Rollins',
 'email': 'gklein@example.com',
 'address': '3898 Werner Mountain\nLake Ruben, AK 36998'}

## Run a local server 

For develpement purposes you generally want to first check your algorithm locally by deploying it on a local http server

In [10]:
config = BeamServeConfig(port=44044, **{'path-to-bundle': '/app/algorithm'})

In [11]:
config

BeamServeConfig->BeamConfig:

Parameters:

batch: null
debug: false
http_backend: waitress
max_batch_size: 10
max_wait_time: 1.0
n_threads: 4
non_blocking: false
path_to_bundle: /app/algorithm
port: 44044
protocol: http
tls: false
use_torch: false

In [12]:
beam_server(fake_alg, protocol=config.protocol, port=config.port, n_threads=config.n_threads,
                use_torch=config.use_torch, batch=config.batch, tls=config.tls,
                max_batch_size=config.max_batch_size, max_wait_time=config.max_wait_time,
                backend=config.http_backend, non_blocking=True)

2024-06-24 07:41:21 | BeamLog | INFO | Opening a flask inference serve on port: 44044


## Client side

Use beam-client to interact with the server

In [13]:
remote_alg = resource('beam-http://localhost:44044')

In [14]:
remote_alg.run(123)

{'name': 'Stephanie Lewis',
 'email': 'edwardferguson@example.com',
 'address': '96003 James Fords\nKimbury, PW 69425'}

Using native requests

In [15]:
import requests

In [26]:
res = requests.post('http://localhost:44044/alg/user/run', json={'args':[123], 'kwargs': {}})

In [29]:
json.loads(res.content.decode('utf-8'))

{'address': '93948 Bonnie Courts\nHoganside, KS 82414',
 'email': 'blaketurner@example.com',
 'name': 'Stephanie Lara'}

Using wget

In [33]:
!wget -qO- http://localhost:44044

{"attributes":{"base_dir":"method","beam_class_name":"property","beam_pickle":"method","clear_cache":"method","error_rate":"variable","excluded_attributes":"variable","fake":"variable","faker_fields":"variable","from_arguments":"method","from_nlp":"method","from_path":"method","from_remote":"method","get_hparam":"method","getattr":"method","hasattr":"method","hparams":"variable","in_beam_pickle":"method","in_cache":"method","init_args_file":"variable","is_initialized":"property","load_state":"method","load_state_dict":"method","name":"property","nlp":"method","run":"method","save_state":"method","save_state_dict":"method","set_hparam":"method","set_name":"method","skeleton_file":"variable","sleep_time":"variable","special_state_attributes":"variable","to_bundle":"method","to_path":"method","update_hparams":"method","variance":"variable"},"hparams":{"debug":false,"llm":null},"name":"obj","obj":"instance","serialization":"pickle","vars_args":["self","sleep_time","variance","error_rate","

In [34]:
!wget -qO- --method POST --header 'Content-Type: application/json' --body-data '{"args": [123]}' http://localhost:44044/alg/user/run

{"address":"1780 Robinson Islands\nJeffreymouth, VI 37853","email":"reyesallison@example.org","name":"Michael Gates"}


# Deploy your algorithm + http server into a docker image

In [49]:
dict(config)

{'debug': False,
 'protocol': 'http',
 'http_backend': 'waitress',
 'path_to_bundle': '/app/algorithm',
 'port': 44044,
 'n_threads': 4,
 'use_torch': False,
 'batch': None,
 'tls': False,
 'max_batch_size': 10,
 'max_wait_time': 1.0,
 'non_blocking': False}

In [51]:
AutoBeam.to_docker(obj=fake_alg, base_image='beam:20240616', image_name='fake-alg-http-server', beam_version='2.5.11', config=config)

2024-06-24 08:35:26 | BeamLog | INFO | Building an object bundle
2024-06-24 08:35:26 | BeamLog | INFO | Saving object's files to path /home/elad/docker/beamds/notebooks/obj: [requirements.json, modules.tar.gz, state, requierements.txt]
2024-06-24 08:35:29 | BeamLog | INFO | Building a Docker image with the requirements and the object bundle. Base image: beam:20240616
Step 1/18 : ARG BASE_IMAGE

Step 2/18 : FROM ${BASE_IMAGE}

---> 726932128776
Step 3/18 : ARG REQUIREMENTS_FILE

---> Running in 48347cd05edc
Removing intermediate container 48347cd05edc
---> 6127762fbf57
Step 4/18 : ARG CONFIG_FILE

---> Running in e156a859ee5a
Removing intermediate container e156a859ee5a
---> 65b185cb9b38
Step 5/18 : ARG ALGORITHM_DIR

---> Running in 001a680db075
Removing intermediate container 001a680db075
---> d958446c5428
Step 6/18 : ARG ENTRYPOINT_SCRIPT

---> Running in 150db15e5d37
Removing intermediate container 150db15e5d37
---> 87c5dd93fbfe
Step 7/18 : ARG BEAM_DS_VERSION

---> Running in 242e8

Run a container with the generated image

In [55]:
!docker run -itd --gpus=all -p 44044:44044 fake-alg-http-server:latest

3df01866773b746e7d83dfb963114b5e07e7edf7de93c017a66ba9b62052763f


In [58]:
alg = resource('beam-http://172.27.0.1:44044')

In [59]:
alg.info

{'attributes': {'base_dir': 'method',
  'beam_class_name': 'property',
  'beam_pickle': 'method',
  'clear_cache': 'method',
  'error_rate': 'variable',
  'excluded_attributes': 'variable',
  'fake': 'variable',
  'faker_fields': 'variable',
  'from_arguments': 'method',
  'from_nlp': 'method',
  'from_path': 'method',
  'from_remote': 'method',
  'get_hparam': 'method',
  'getattr': 'method',
  'hasattr': 'method',
  'hparams': 'variable',
  'in_beam_pickle': 'method',
  'in_cache': 'method',
  'init_args_file': 'variable',
  'is_initialized': 'property',
  'is_notebook': 'variable',
  'llm': 'variable',
  'load_state': 'method',
  'load_state_dict': 'method',
  'name': 'property',
  'nlp': 'method',
  'run': 'method',
  'save_state': 'method',
  'save_state_dict': 'method',
  'set_hparam': 'method',
  'set_name': 'method',
  'skeleton_file': 'variable',
  'sleep_time': 'variable',
  'special_state_attributes': 'variable',
  'to_bundle': 'method',
  'to_path': 'method',
  'update_hpar

In [60]:
alg.run(123)

{'name': 'Mrs. Victoria Chaney',
 'email': 'sarah44@example.net',
 'address': '8436 Wright Viaduct Suite 933\nLake Clayton, NH 16942'}